Custom transformers
---

In [1]:
import pandas as pd

data_df = pd.read_csv('messy-bikes.csv')
data_df.head()

,temp,hum,windspeed,yr,workingday,holiday,weekday,season,weathersit,casual
0,0.344,0.806,NaN,2011.0,no,NaN,6.0,spring,cloudy,331
1,0.363,0.696,0.249,2011.0,NaN,no,0.0,spring,cloudy,131
2,0.196,0.437,0.248,2011.0,yes,no,NaN,NaN,clear,120
3,NaN,0.590,0.160,2011.0,yes,no,2.0,spring,NaN,108
4,0.227,0.437,0.187,2011.0,yes,no,3.0,spring,clear,82


In [2]:
data_df.isnull().mean()

temp          0.095759
hum           0.109439
windspeed     0.112175
yr            0.087551
workingday    0.088919
holiday       0.102599
weekday       0.097127
season        0.102599
weathersit    0.103967
casual        0.000000
dtype: float64

In [3]:
import numpy as np

def preprocess_f(df):
    # Work on a copy
    df = df.copy()
    
    # Missing values in continuous features
    cont_vars = ['temp', 'hum', 'windspeed']
    for c in cont_vars:
        df[c] = df[c].fillna(df[c].mean()) # replace by mean
        
    # Explicitely convert to string values
    to_convert = ['yr', 'weekday']
    convert_f = lambda x: str(int(x)) if not np.isnan(x) else np.nan
    df[to_convert] = df[to_convert].applymap(convert_f)
    
    # .. in categorical ones: create 'missing' category
    cat_vars = ['yr', 'workingday', 'holiday', 'weekday', 'season', 'weathersit']
    df[cat_vars] = df[cat_vars].fillna('missing')
    
    # One-hot encoding
    df = pd.get_dummies(df)
    
    return df
    
preprocessed = preprocess_f(data_df)
preprocessed.head()

,temp,hum,windspeed,casual,yr_2011,yr_2012,yr_missing,workingday_missing,workingday_no,workingday_yes,...,weekday_missing,season_fall,season_missing,season_spring,season_summer,season_winter,weathersit_clear,weathersit_cloudy,weathersit_missing,weathersit_rainy
0,0.344000,0.806,0.188969,331,1,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
1,0.363000,0.696,0.249000,131,1,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
2,0.196000,0.437,0.248000,120,1,0,0,0,0,1,...,1,0,1,0,0,0,1,0,0,0
3,0.495543,0.590,0.160000,108,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
4,0.227000,0.437,0.187000,82,1,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,0


In [4]:
preprocessed.columns

Index(['temp', 'hum', 'windspeed', 'casual', 'yr_2011', 'yr_2012',
       'yr_missing', 'workingday_missing', 'workingday_no', 'workingday_yes',
       'holiday_missing', 'holiday_no', 'holiday_yes', 'weekday_0',
       'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5',
       'weekday_6', 'weekday_missing', 'season_fall', 'season_missing',
       'season_spring', 'season_summer', 'season_winter', 'weathersit_clear',
       'weathersit_cloudy', 'weathersit_missing', 'weathersit_rainy'],
      dtype='object')

In [7]:
from sklearn.preprocessing import FunctionTransformer

preprocessor = FunctionTransformer(preprocess_f, validate=False) # validate = To avoid conversion from DataFram to Numpy array
preprocessed = preprocessor.fit_transform(data_df)
preprocessed.head()

,temp,hum,windspeed,casual,yr_2011,yr_2012,yr_missing,workingday_missing,workingday_no,workingday_yes,...,weekday_missing,season_fall,season_missing,season_spring,season_summer,season_winter,weathersit_clear,weathersit_cloudy,weathersit_missing,weathersit_rainy
0,0.344000,0.806,0.188969,331,1,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
1,0.363000,0.696,0.249000,131,1,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
2,0.196000,0.437,0.248000,120,1,0,0,0,0,1,...,1,0,1,0,0,0,1,0,0,0
3,0.495543,0.590,0.160000,108,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
4,0.227000,0.437,0.187000,82,1,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,0


In [8]:
preprocessor.transform(data_df.iloc[:1])

,temp,hum,windspeed,casual,yr_2011,workingday_no,holiday_missing,weekday_6,season_spring,weathersit_cloudy
0,0.344,0.806,NaN,331,1,1,1,1,1,1


In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

class PandasPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, preprocess_f):
        self.preprocess_f = preprocess_f

    def fit(self, X_df, y=None):
        # Check that we get a DataFrame
        assert type(X_df) == pd.DataFrame
        
        # Preprocess data
        X_preprocessed = self.preprocess_f(X_df)
        
        # Save columns names/order for inference time
        self.columns_ = X_preprocessed.columns
        
        return self

    def transform(self, X_df):
        # Check that we get a DataFrame
        assert type(X_df) == pd.DataFrame
        
        # Preprocess data
        X_preprocessed = self.preprocess_f(X_df)
        
        # Make sure to have the same features
        X_reindexed = X_preprocessed.reindex(columns=self.columns_, fill_value=0)
        
        return X_reindexed
    
preprocessor = PandasPreprocessor(preprocess_f)
preprocessor.fit(data_df)
preprocessor.transform(data_df.iloc[:1])

,temp,hum,windspeed,casual,yr_2011,yr_2012,yr_missing,workingday_missing,workingday_no,workingday_yes,...,weekday_missing,season_fall,season_missing,season_spring,season_summer,season_winter,weathersit_clear,weathersit_cloudy,weathersit_missing,weathersit_rainy
0,0.344,0.806,NaN,331,1,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0


In [10]:
class PandasPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.cat_vars_ = ['yr', 'workingday', 'holiday', 'weekday', 'season', 'weathersit']
        self.cont_vars_ = ['temp', 'hum', 'windspeed']
        self.to_convert_ = ['yr', 'weekday']
    
    def preprocess_f(self, X_df, train_mean):
        # Work on a copy
        X_df = X_df.copy()
        
        # Missing values in continuous features
        for c in self.cont_vars_:
            X_df[c] = X_df[c].fillna(train_mean[c])
        
        # Explicitely convert to string values
        convert_f = lambda x: str(int(x)) if not np.isnan(x) else np.nan
        X_df[self.to_convert_] = X_df[self.to_convert_].applymap(convert_f)
    
        # .. in categorical ones: create 'missing' category
        X_df[self.cat_vars_] = X_df[self.cat_vars_].fillna('missing')
    
        # One-hot encoding
        X_df = pd.get_dummies(X_df)
        
        return X_df
        
    def fit(self, X_df, y=None):
        # Check that we get a DataFrame
        assert type(X_df) == pd.DataFrame
    
        # Save train mean for continuous variables
        self.train_mean_ = X_df[self.cont_vars_].mean() 
        
        # Preprocess data
        X_preprocessed = self.preprocess_f(X_df, self.train_mean_)
        
        # Save columns names/order for inference time
        self.columns_ = X_preprocessed.columns 
        
        return self

    def transform(self, X_df):
        # Check that we get a DataFrame
        assert type(X_df) == pd.DataFrame
        
        # Preprocess data
        X_preprocessed = self.preprocess_f(X_df, self.train_mean_)
        
        # Make sure to have the same features
        X_reindexed = X_preprocessed.reindex(columns=self.columns_, fill_value=0)
        
        return X_reindexed
    
preprocessor = PandasPreprocessor()
preprocessor.fit(data_df)
preprocessor.transform(data_df.iloc[:1])

,temp,hum,windspeed,casual,yr_2011,yr_2012,yr_missing,workingday_missing,workingday_no,workingday_yes,...,weekday_missing,season_fall,season_missing,season_spring,season_summer,season_winter,weathersit_clear,weathersit_cloudy,weathersit_missing,weathersit_rainy
0,0.344,0.806,0.188969,331,1,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0


In [11]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# Use our custom transformer in a pipeline
pipe = Pipeline([
    ('preprocessor', PandasPreprocessor()),
    ('estimator', LinearRegression())
])

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as MAE

# Split data
X = data_df.drop('casual', axis=1)
y = data_df.casual
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3, random_state=0)

# Evaluate estimator
pipe.fit(X_tr, y_tr)
print('MAE: {:.0f}'.format(MAE(y_te, pipe.predict(X_te))))

MAE: 293
